<a href="https://colab.research.google.com/github/kadiyalamani15/AQI-VAR/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Read every months data by traversing the folders
# Upsample them to 15 min freq
# Append to the year dataframe [Ex: df_stationName_year]
# For each station, four dataframes corresponding to each year should be present

In [ ]:
# loop over every station, every year, delete the first 11 rows and pass the data to upsample function
# the upsampled data should be appended to the main dataframe

In [1]:
# Import required Libraries

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
import os
from os import listdir

In [5]:
for station in sorted(os.listdir(f"raw_data")):
  station_year_df = pd.DataFrame()
  if station == "1600":
    factors = 6
  elif station == "1601":
    factors = 11
  else:
    continue
  print(f"Station: {station}, factors: {factors}")
  for year in sorted(os.listdir(f"raw_data/{station}")):
    if not year.startswith('.'):
      station_year_df = pd.DataFrame()
      for month in sorted(os.listdir(f"raw_data/{station}/{year}")):
        if not month.startswith('.'):
          print(f"Processing raw_data/{station}/{year}/{month} file ...")
          # Call the data cleaning function
          c = pd.read_excel(f"raw_data/{station}/{year}/{month}")
          d = c.iloc[11:,:factors+1]
          #print(c.iloc[10:11,:factors+1].values[0])
          d.columns = c.iloc[10:11,:factors+1].values[0]

          try:
            d[['date','time']] = d['Date & Time'].str.split(' ', expand=True)
          except ValueError as NoDataError:
            print(f"No Data recorded in file: {station}-{year}-{month}")
          else:
            d.iloc[:,-2] = pd.to_datetime(d.iloc[:,-2])

            for i in range(len(d)):
              if "24:00" in d.iloc[i,-1]:
                #print(d.iloc[i,-2].month)
                rev_day = d.iloc[i,-2].day + 1
                try:
                  d.iloc[i,-2].replace(day=rev_day)
                except ValueError  as dayError:
                  print("Month end error!")
                  rev_month = d.iloc[i,-2].month + 1
                  try:
                    d.iloc[i,-2].replace(day=1, month=rev_month)
                  except ValueError as yearError:
                    print("Year end error!")
                    rev_year = d.iloc[i,-2].year + 1
                    d.iloc[i,-2] = d.iloc[i,-2].replace(day=1, month=1, year=rev_year)
                    d.iloc[i,-1] = d.iloc[i,-1].replace("24:00","00:00")
                  else:
                    d.iloc[i,-2] = d.iloc[i,-2].replace(day=1, month=rev_month)
                    d.iloc[i,-1] = d.iloc[i,-1].replace("24:00","00:00")
                # End of the month
                else:
                  d.iloc[i,-2] = d.iloc[i,-2].replace(day=rev_day)
                  d.iloc[i,-1] = d.iloc[i,-1].replace("24:00","00:00")
                  #print(d.iloc[i,-2].replace(day= rev_day))
                
              else:
                continue

            d['Date & Time'] = d['date'].astype(str) + ' ' + d['time']
            d.drop(['date','time'], axis=1, inplace=True)

          for j in range(0, d.shape[1]):
                d.iloc[:,j] = d.iloc[:,j].str.rstrip('(U)')

          
          d = d.replace(regex=['(D)','(F)','(I)','(M)','(C)','(N)','(W)','(G)','(Z)'], value=np.nan)
          station_year_df = station_year_df.append(d)
          #break
        else:
          continue
      # resample, save as one df
      station_year_df = station_year_df.set_index('Date & Time')
      station_year_df.index = pd.DatetimeIndex(station_year_df.index)
      station_year_df = station_year_df.resample('15T').asfreq()

      # Save year data to excel
      station_year_df.to_excel(f"{station}_{year}.xlsx")
      print(f"\nSaving {station} station's {year} year processed data to excel ...\n")
      #break
      #station_year_df = pd.DataFrame()
    else:
      continue

Station: 1600, factors: 6
Processing raw_data/1600/2019/01.xlsx file ...
Month end error!
Processing raw_data/1600/2019/02.xlsx file ...
Month end error!
Processing raw_data/1600/2019/03.xlsx file ...
Processing raw_data/1600/2019/04.xlsx file ...
Month end error!
Processing raw_data/1600/2019/05.xlsx file ...
Processing raw_data/1600/2019/06.xlsx file ...
Month end error!
Processing raw_data/1600/2019/07.xlsx file ...
Month end error!
Processing raw_data/1600/2019/08.xlsx file ...
Processing raw_data/1600/2019/09.xlsx file ...
Processing raw_data/1600/2019/10.xlsx file ...
Processing raw_data/1600/2019/11.xlsx file ...
Month end error!
Processing raw_data/1600/2019/12.xlsx file ...
Month end error!
Year end error!

Saving 1600 station's 2019 year processed data to excel ...

Processing raw_data/1600/2020/01.xlsx file ...
Month end error!
Processing raw_data/1600/2020/02.xlsx file ...
Month end error!
Processing raw_data/1600/2020/03.xlsx file ...
Month end error!
Processing raw_data/1